### Dataset

Lead scoring dataset Bank Marketing dataset. [Download](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv).

With `wget`:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
```
> I downloaded in the terminal to my ../data folder

In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.

### Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For categorical features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

df = pd.read_csv('../data/course_lead_scoring.csv')

(1462, 9)

In [ ]:
df.head()
#df.shape

#data already looks neat with lowercase, underscores, etc
#checking for missing values
df.isnull().sum()


lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [ ]:
#split categorical and numerical columns for different NA processing

df.dtypes

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
numerical_columns = list(df.dtypes[df.dtypes != 'object'].index)

df[categorical_columns].isnull().sum()
df[numerical_columns].isnull().sum()    

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
converted                     0
dtype: int64